In [8]:
import os
import json
from tqdm import tqdm
import csv
from random import shuffle

In [2]:
data_dir = "./test/"
cot_dir = "./val/"

In [12]:
file_list = os.listdir(data_dir)
all_data = list()
for file_name in tqdm(file_list):
    if file_name in ["README.md", ".ipynb_checkpoints"]:
        continue
    task_name = file_name.replace("_test.csv", "")
    task_data = list()
    with open(os.path.join(data_dir, file_name), "r", encoding="utf-8") as fr:
        csv_reader = csv.reader(fr)
        for row in csv_reader:  # 将csv 文件中的数据保存到birth_data中
            # birth_data.append(row)
            task_data.append({
                "instruction": "Q: {}\nOptions:\n(A): {}\n(B): {}\n(C): {}\n(D): {}".format(row[0], row[1], row[2], row[3], row[4]),
                "answer": "({})".format(row[5])
            })

    cot_examples = list()
    with open(os.path.join(cot_dir, "{}_val.csv".format(task_name)), "r", encoding="utf-8") as fr:
        csv_reader = csv.reader(fr)
        for row in csv_reader:  # 将csv 文件中的数据保存到birth_data中
            # birth_data.append(row)
            cot_examples.append(
                "Q: {}\nOptions:\n(A): {}\n(B): {}\n(C): {}\n(D): {}\nA: The answer is ({}).".format(row[0], row[1], row[2], row[3], row[4], row[5])
            )
    shuffle(cot_examples)
    cot_prompt = "\n\n".join(cot_examples[:5])
     
    for ei, example in enumerate(task_data):
        all_data.append({
            "task_name": task_name,
            "idx": ei + 1,
            "instruction": "{}\n\n{}\nA:".format(cot_prompt, example["instruction"]),
            "answer": [example["answer"]],
        })

100%|██████████| 57/57 [00:00<00:00, 522.23it/s]


In [14]:
print(all_data[1400]["instruction"])
print(all_data[1400]["answer"])

Q: What is soft power?
Options:
(A): Use of coercive measures short of war, such as economic sanctions
(B): Ways in which the values and culture of a country provide it with influence in the international system
(C): Ineffective use of one's power
(D): When states use carrots (incentives) instead of sticks (threats) to influence other countries.
A: The answer is (B).

Q: Which factor is cited by Revisionists as the primary cause of the Cold War?
Options:
(A): The threat posed by the Soviet Union
(B): Domestic concerns of the US
(C): Soviet ideology
(D): None of the above
A: The answer is (B).

Q: According to Realists, what accounts for the onset of the Cold War?
Options:
(A): Ideological differences
(B): A power vacuum
(C): The expansionist nature of the Soviet Union
(D): Both b and c
A: The answer is (D).

Q: Who is considered the “father” of containment?
Options:
(A): George Kennan
(B): John Foster Dulles
(C): Henry Kissinger
(D): Dwight Eisenhower
A: The answer is (A).

Q: Who were

In [15]:
# save data
with open("mmlu_dataset.json", "w", encoding="utf-8") as fw:
    for example in tqdm(all_data):
        fw.write(json.dumps(example) + "\n")

100%|██████████| 14042/14042 [00:00<00:00, 61485.38it/s]
